In [1]:
import sys
sys.path.insert(0, '../')
import picea
picea.__version__,picea

('0.0.3_beta1', <module 'picea' from '../picea/__init__.py'>)

In [2]:
from picea import SequenceCollection

In [3]:
s = SequenceCollection()

seq1 = 'ACGTAGCAG'
seq2 = 'CCGTCGCAGA'
seq3 = 'ACGTAG-CA'

for seq in [seq1,seq2,seq3]:
    s['s'] = seq
s

../picea/sequence.py:40: UserWarning: Turning duplicate header "s" into unique header
  warn(f'Turning duplicate header "{header}" into unique header')


In [4]:
s.headers

['s', 's_1', 's_2']

In [5]:
s.sequences

['ACGTAGCAG', 'CCGTCGCAGA', 'ACGTAG-CA']

In [6]:
s.is_aligned = False
s.sequences

['ACGTAGCAG', 'CCGTCGCAGA', 'ACGTAG-CA']

In [7]:
s = SequenceCollection.from_fasta(string='>s\nACGT\n>s1\najscjasc')
s.headers,s.sequences,s._collection

(['s', 's1'],
 ['ACGT', 'ajscjasc'],
 array([[ 65,  67,  71,  84,  45,  45,  45,  45],
        [ 97, 106, 115,  99, 106,  97, 115,  99]], dtype=uint8))

In [9]:
s = SequenceCollection()
s['s1'] = 'akj'
print(s.headers,s.sequences,s._collection)
s['s2'] = 'ashdajsdlajsjdljkasldjlasjkd'
print(s.headers,s.sequences,s._collection)

['s1'] ['akj'] [[ 97 107 106]]
['s1', 's2'] ['akj', 'ashdajsdlajsjdljkasldjlasjkd'] [[ 97 107 106  45  45  45  45  45  45  45  45  45  45  45  45  45  45  45
   45  45  45  45  45  45  45  45  45  45]
 [ 97 115 104 100  97 106 115 100 108  97 106 115 106 100 108 106 107  97
  115 108 100 106 108  97 115 106 107 100]]


In [10]:
fasta = '>A\nABC\n>B\nDEF'
seq = SequenceCollection.from_fasta(string=fasta)
seq.to_fasta() == fasta

True